In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2023
path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"

table = pq.read_table(path)
rides = table.to_pandas()
rides.iloc[1000:1020]

,pickup_datetime,pickup_location_id
1000,2023-01-30 18:16:08,HB607
1001,2023-01-28 16:18:04,HB607
1002,2023-01-02 16:23:07,HB607
1003,2023-01-24 10:22:18,HB607
1004,2023-01-26 10:17:34,HB607
1005,2023-01-10 20:09:56,HB607
1006,2023-01-28 20:19:08,JC109
1007,2023-01-12 00:20:44,JC002
1008,2023-01-05 12:25:58,JC002
1009,2023-01-11 20:49:12,HB402


In [2]:
rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55748 entries, 0 to 55747
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_datetime     55748 non-null  datetime64[ns]
 1   pickup_location_id  55748 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 871.2+ KB


In [3]:
# 2. Add hourly bucket
rides["pickup_datetime"] = pd.to_datetime(rides["pickup_datetime"])
rides["pickup_hour"] = rides["pickup_datetime"].dt.floor('h')

In [5]:

# 3. Aggregate rides per hour per location
agg_rides = rides.groupby(["pickup_hour", "pickup_location_id"]).size().reset_index(name="rides")
agg_rides.tail()


,pickup_hour,pickup_location_id,rides
25133,2023-01-31 23:00:00,JC023,1
25134,2023-01-31 23:00:00,JC052,1
25135,2023-01-31 23:00:00,JC082,1
25136,2023-01-31 23:00:00,JC109,1
25137,2023-01-31 23:00:00,JC115,6


In [6]:
# 4. Fill missing hour-location pairs with 0 rides
def fill_missing_rides_full_range(df, hour_col, location_col, rides_col):
    df[hour_col] = pd.to_datetime(df[hour_col])
    full_hours = pd.date_range(df[hour_col].min(), df[hour_col].max(), freq="h")
    all_locations = df[location_col].unique()
    full_combinations = pd.DataFrame(
        [(hour, loc) for hour in full_hours for loc in all_locations],
        columns=[hour_col, location_col]
    )
    merged_df = pd.merge(full_combinations, df, on=[hour_col, location_col], how='left')
    merged_df[rides_col] = merged_df[rides_col].fillna(0).astype(int)
    return merged_df

agg_data_filled = fill_missing_rides_full_range(
    agg_rides, hour_col="pickup_hour", location_col="pickup_location_id", rides_col="rides"
).sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)


In [16]:
# 5. Save to Parquet
output_path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"
output_path.parent.mkdir(parents=True, exist_ok=True)
agg_data_filled.to_parquet(output_path, engine="pyarrow", index=False)

# Optional: preview the result
agg_data_filled.head()

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,5532.01,0
1,2023-01-01 01:00:00,5532.01,0
2,2023-01-01 02:00:00,5532.01,0
3,2023-01-01 03:00:00,5532.01,0
4,2023-01-01 04:00:00,5532.01,0


In [18]:
import pandas as pd
from pathlib import Path

# Load the dataset
year = 2023
month = 1
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"
rides = pd.read_parquet(path)

# Remove any rows where pickup_location_id is not alphanumeric (e.g., '5532.01')
rides = rides[rides["pickup_location_id"].apply(lambda x: isinstance(x, str) and x.isalnum())]

# Save cleaned data back (optional)
output_path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"
rides.to_parquet(output_path, index=False)

# Confirm
print("Cleaned dataset saved. Unique location IDs:")
print(sorted(rides["pickup_location_id"].unique()))


Cleaned dataset saved. Unique location IDs:
['HB101', 'HB102', 'HB103', 'HB105', 'HB201', 'HB202', 'HB203', 'HB301', 'HB302', 'HB303', 'HB304', 'HB305', 'HB401', 'HB402', 'HB404', 'HB407', 'HB408', 'HB409', 'HB501', 'HB502', 'HB503', 'HB505', 'HB506', 'HB507', 'HB601', 'HB602', 'HB603', 'HB607', 'JC002', 'JC003', 'JC006', 'JC008', 'JC009', 'JC013', 'JC014', 'JC018', 'JC019', 'JC020', 'JC022', 'JC023', 'JC024', 'JC027', 'JC032', 'JC034', 'JC035', 'JC038', 'JC051', 'JC052', 'JC053', 'JC055', 'JC057', 'JC059', 'JC063', 'JC065', 'JC066', 'JC072', 'JC074', 'JC075', 'JC076', 'JC077', 'JC078', 'JC080', 'JC081', 'JC082', 'JC084', 'JC093', 'JC094', 'JC095', 'JC097', 'JC098', 'JC099', 'JC102', 'JC103', 'JC104', 'JC105', 'JC106', 'JC107', 'JC108', 'JC109', 'JC110', 'JC115']


,pickup_datetime,pickup_location_id
0,2023-01-28 09:18:10,HB101
1,2023-01-23 20:10:12,HB101
2,2023-01-29 15:27:04,HB101
3,2023-01-24 18:35:08,HB101
4,2023-01-21 20:44:09,JC009
